In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont

In [2]:
test_final_100k_ = pd.read_csv('Datos/test_final_100k.csv')

# Pre procesamiento de los datos


## Postulantes

In [3]:
postulantes_educacion_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_educacion_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_educacion_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

In [4]:
postulantes_genero_y_edad = pd.concat([postulantes_genero_y_edad_desde_15_Abril,postulantes_genero_y_edad_hasta_15_abril,postulantes_genero_y_edad_original], axis=0)
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates('idpostulante')

In [5]:
#Descarto los postulantes que están presentes en el set de prueba

postulantes_genero_y_edad = postulantes_genero_y_edad.set_index('idpostulante').drop(test_final_100k_.drop_duplicates('idpostulante').idpostulante).reset_index()

In [6]:
postulantes_educacion = pd.concat([postulantes_educacion_desde_15_Abril,postulantes_educacion_hasta_15_abril,postulantes_educacion_original], axis=0)
postulantes_educacion = postulantes_educacion

In [7]:
#Renombro las columnas a un nombre más apropiado

postulantes_educacion = postulantes_educacion.rename(columns={"nombre":"educacion"})

In [8]:
#Hay algun dato nulo?

postulantes_educacion.isnull().any()

idpostulante    False
educacion       False
estado          False
dtype: bool

In [9]:
postulantes_educacion.estado.value_counts()

Graduado      721794
En Curso      286282
Abandonado     95117
Name: estado, dtype: int64

In [10]:
#Para cada postulante, me interesa quedarme solamente con la mayor de sus educaciones

postulantes_educacion['Cod_edu'] = np.array([0 if e == 'Secundario' else 
                                             1 if e == 'Otro' else 
                                             2 if e == 'Terciario/Técnico' else 
                                             3 if e == 'Universitario' else 
                                             4 if e == 'Posgrado' else 
                                             5 if e == 'Master' else 6 for e in postulantes_educacion.educacion])

postulantes_educacion['Cod_est'] = np.array([1 if e == 'Abandonado' else 
                                             2 if e == 'En curso' else 3 for e in postulantes_educacion.estado])

postulantes_educacion = postulantes_educacion.sort_values(['idpostulante','Cod_edu'], ascending = False).drop_duplicates('idpostulante')

In [11]:
postulantes_educacion.head()

,idpostulante,educacion,estado,Cod_edu,Cod_est
53093,zvxzzL6,Otro,Graduado,1,3
118722,zvxzvD5,Terciario/Técnico,En Curso,2,3
60154,zvxzrWm,Universitario,Graduado,3,3
50863,zvxzmED,Secundario,Graduado,0,3
211836,zvxzjoK,Terciario/Técnico,En Curso,2,3


In [12]:
postulantes_genero_y_edad['Cod_sexo'] = np.array([0 if e == 'MASC' else 1 for e in postulantes_genero_y_edad.sexo])

In [13]:
#Ojo, fechas de nacimiento nulas

postulantes_genero_y_edad.isnull().any()

idpostulante       False
fechanacimiento     True
sexo               False
Cod_sexo           False
dtype: bool

In [14]:
#Convierto la columna fechanacimiento a formato datetime. 

postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'], errors='coerce')

In [15]:
#Corregir estas fechas nulas

postulantes_genero_y_edad.loc[postulantes_genero_y_edad.fechanacimiento.isnull()].head()

,idpostulante,fechanacimiento,sexo,Cod_sexo
1,8mW,NaT,NO_DECLARA,1
5,eRYllW,NaT,NO_DECLARA,1
6,52KB1p,NaT,NO_DECLARA,1
11,Y0mpjq,NaT,NO_DECLARA,1
14,a61YjW,NaT,NO_DECLARA,1


In [16]:
#Vamos ahora a agregar una columna adicional al dataframe, de modo que podamos observar la edad de cada persona

postulantes_genero_y_edad['edad'] = ((pd.datetime.today() - postulantes_genero_y_edad['fechanacimiento']).dt.days/365)
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].fillna(0)
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype(int)
postulantes_genero_y_edad.head(10)

,idpostulante,fechanacimiento,sexo,Cod_sexo,edad
0,YZP,1961-05-01,MASC,0,57
1,8mW,NaT,NO_DECLARA,1,0
2,aEWON,1967-06-16,FEM,1,50
3,Zjde31,1964-02-08,MASC,0,54
4,8wKqoM,1977-08-04,FEM,1,40
5,eRYllW,NaT,NO_DECLARA,1,0
6,52KB1p,NaT,NO_DECLARA,1,0
7,NA8o45,1974-08-16,MASC,0,43
8,eRYl5r,1975-10-11,FEM,1,42
9,8wKOlR,1977-12-24,MASC,0,40


In [17]:
#Reviso la edad de las personas que cumplen años el 29/02

postulantes_nacidos_en_febrero = postulantes_genero_y_edad.loc[postulantes_genero_y_edad['fechanacimiento'].dt.month == 2]
postulantes_nacidos_en_febrero.loc[postulantes_nacidos_en_febrero['fechanacimiento'].dt.day == 29].sort_values('fechanacimiento',ascending = False).head(10)

,idpostulante,fechanacimiento,sexo,Cod_sexo,edad
11375,VNr1dej,2000-02-29,MASC,0,18
437559,6rP38R3,2000-02-29,FEM,1,18
435820,3NP3dAJ,2000-02-29,FEM,1,18
431417,Bm5dkzR,2000-02-29,MASC,0,18
180285,96XkKlw,2000-02-29,FEM,1,18
138853,Yj5p1GX,2000-02-29,MASC,0,18
148087,1QP82xR,2000-02-29,FEM,1,18
197610,EzeN2Q4,2000-02-29,FEM,1,18
355768,JBraa0E,2000-02-29,MASC,0,18
333738,ekOwAWW,2000-02-29,FEM,1,18


In [18]:
#Merge de los datos de los postulantes

postulantes = pd.merge(postulantes_genero_y_edad,postulantes_educacion,how='left',on='idpostulante')
postulantes['educacion'] = postulantes['educacion'].fillna('Ninguna')
postulantes['estado'] = postulantes['estado'].fillna('Ninguno')
postulantes['Cod_est'] = postulantes['Cod_est'].fillna('0')
postulantes = postulantes.drop(['fechanacimiento'],axis=1)
postulantes.head()

,idpostulante,sexo,Cod_sexo,edad,educacion,estado,Cod_edu,Cod_est
0,YZP,MASC,0,57,Master,En Curso,5.0,3
1,8mW,NO_DECLARA,1,0,Ninguna,Ninguno,NaN,0
2,aEWON,FEM,1,50,Ninguna,Ninguno,NaN,0
3,Zjde31,MASC,0,54,Posgrado,Graduado,4.0,3
4,8wKqoM,FEM,1,40,Universitario,Graduado,3.0,3


In [19]:
postulantes.isnull().any()

idpostulante    False
sexo            False
Cod_sexo        False
edad            False
educacion       False
estado          False
Cod_edu          True
Cod_est         False
dtype: bool

## Vistas

In [20]:
#vistas = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_3_vistas.csv')

In [21]:
#Vistazo a la estructura del dataframe

#vistas.head()

In [22]:
#El dataframe posee 5758686 registros

#vistas.shape

In [23]:
#No tenemos registros nulos

#vistas.isnull().any()

In [24]:
#En este dataframe tenemos 11223 avisos únicos 

#vistas['idAviso'].value_counts().size

In [25]:
#Y de los postulantes del dataset, 272526 han visto al menos un aviso

#vistas['idpostulante'].value_counts().size

In [26]:
#Modifico el nombre de la columna timestamp y en su lugar pongo un nombre más descriptivo.>

#vistas = vistas.rename(columns = {'timestamp':'fechahoravisualizacion'})

In [27]:
#Ordeno la informacion sobre la fecha y hora de visualizaciones, modificando su formato y agregando columnas que me permitan
#analizar de forma individual la fecha y la hora.

#vistas['fechahoravisualizacion'] = pd.to_datetime(vistas['fechahoravisualizacion'])
#vistas['fechavisualizacion'] = vistas['fechahoravisualizacion'].dt.date
#vistas['horavisualizacion'] = vistas['fechahoravisualizacion'].dt.time
#vistas.sort_values('fechavisualizacion').head(10)

## Postulaciones

In [28]:
postulaciones_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv')
postulaciones_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_4_postulaciones.csv')

In [29]:
postulaciones = pd.concat([postulaciones_hasta_15_abril,postulaciones_original],axis=0).drop_duplicates()

In [30]:
#Vistazo a la estructura del dataframe

postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [31]:
#Tenemos 6604534 registros

postulaciones.shape

(6604534, 3)

In [32]:
#No hay registros nulos

postulaciones.isnull().any()

idaviso             False
idpostulante        False
fechapostulacion    False
dtype: bool

## Avisos

In [33]:
#avisos_detalle_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_6_avisos_detalle.csv')
#avisos_online_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv')
#avisos_detalle_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv')
#avisos_online_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_5_avisos_online.csv')
#avisos_detalle_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
#avisos_detalle2 = pd.read('fiuba_6_avisos_detalle_missing_nivel_laboral')

In [34]:
#avisos_detalle = pd.concat([avisos_detalle2,avisos_detalle_desde_15_Abril,avisos_detalle_hasta_15_abril,avisos_detalle_original],axis=0).drop_duplicates('idaviso')
#avisos_online = pd.concat([avisos_online_hasta_15_abril,avisos_online_original],axis=0).drop_duplicates('idaviso')

In [35]:
#avisos_online.shape

In [36]:
#avisos_detalle.shape

In [37]:
#No hay registros nulos

#avisos_online.isnull().any()

In [38]:
#avisos_detalle.isnull().any()

# Árbol binario

In [39]:
#Todas las postulaciones historicas

postulaciones_historico = pd.merge(postulantes,postulaciones[['idaviso','idpostulante']],on='idpostulante',how='inner')

In [40]:
postulaciones_historico.Cod_edu = postulaciones_historico.Cod_edu.fillna(7)
postulaciones_historico = postulaciones_historico.drop(['idpostulante','sexo','educacion','estado'],axis=1)
postulaciones_historico.head()

,Cod_sexo,edad,Cod_edu,Cod_est,idaviso
0,0,35,3.0,3,1112261798
1,0,35,3.0,3,1112269651
2,0,35,3.0,3,1112341049
3,0,35,3.0,3,1112259997
4,0,35,3.0,3,1112352499


In [41]:
y_train = postulaciones_historico['idaviso']
x_train = postulaciones_historico.drop(['idaviso'], axis=1).values 

In [42]:
x_train

array([[0, 35, 3.0, 3.0],
       [0, 35, 3.0, 3.0],
       [0, 35, 3.0, 3.0],
       ...,
       [1, 20, 2.0, 3.0],
       [1, 20, 2.0, 3.0],
       [1, 0, 7.0, '0']], dtype=object)

In [43]:
y_train.head()

0    1112261798
1    1112269651
2    1112341049
3    1112259997
4    1112352499
Name: idaviso, dtype: int64

In [48]:
cv = KFold(n_splits=10) # Numero deseado de "folds" que haremos
accuracies = list()
max_attributes = len(list(postulaciones_historico))
depth_range = range(1, max_attributes + 1)

# Testearemos la profundidad de 1 a cantidad de atributos +1
for depth in depth_range:
    fold_accuracy = []
    tree_model = tree.DecisionTreeClassifier(criterion='entropy',
                                             min_samples_split=20,
                                             min_samples_leaf=5,
                                             max_depth = depth)
    for train_fold, valid_fold in cv.split(postulaciones_historico):
        f_train = postulaciones_historico.loc[train_fold] 
        f_valid = postulaciones_historico.loc[valid_fold] 

        model = tree_model.fit(X = f_train[0:5000].drop(['idaviso'], axis=1), 
                               y = f_train[0:5000]["idaviso"]) 
        valid_acc = model.score(X = f_valid[0:5000].drop(['idaviso'], axis=1), 
                                y = f_valid[0:5000]["idaviso"]) # calculamos la precision con el segmento de validacion
        fold_accuracy.append(valid_acc)

    avg = sum(fold_accuracy)/len(fold_accuracy)
    accuracies.append(avg)
    
# Mostramos los resultados obtenidos
df = pd.DataFrame({"Max Depth": depth_range, "Average Accuracy": accuracies})
df = df[["Max Depth", "Average Accuracy"]]
print(df.to_string(index=False))

Max Depth  Average Accuracy
        1           0.00354
        2           0.00336
        3           0.00316
        4           0.00252
        5           0.00170


In [49]:
#Crear Arbol de decision con profundidad = 1
decision_tree = tree.DecisionTreeClassifier(criterion='entropy',
                                            min_samples_split=20,
                                            min_samples_leaf=5,
                                            max_depth = 1)
decision_tree = decision_tree.fit(x_train, y_train)

In [58]:
y_pred = decision_tree.predict(x_train[0:1])
print("Prediccion: " + str(y_pred))


Prediccion: [1112033906]


In [61]:
acc_decision_tree = round(decision_tree.score(x_train[0:5000], y_train[0:5000]) * 100, 2)
print(acc_decision_tree)

0.22
